# BLEU评分学习笔记

## 目录
1. [BLEU评分简介](#1-bleu评分简介)
2. [BLEU评分的数学原理](#2-bleu评分的数学原理)
3. [BLEU评分的计算步骤](#3-bleu评分的计算步骤)

## 1. BLEU评分简介

###  什么是BLEU评分？
**BLEU (Bilingual Evaluation Understudy)** 是一种自动评估机器翻译质量的指标，由IBM研究院于2002年提出。BLEU评分通过比较机器翻译结果与一个或多个参考翻译之间的相似度来评估翻译质量。

### BLEU的基本思想
BLEU的核心思想是：**好的翻译应该与人工参考翻译在用词和表达上有较高的重叠度**。具体来说，BLEU主要考察：

- **n-gram精确率**：机器翻译中的n-gram在参考翻译中出现的比例
- **长度惩罚**：防止系统生成过短的翻译以获得高精确率

## 2. BLEU评分的数学原理

###  n-gram精确率
n-gram精确率是BLEU的核心组成部分，计算公式如下：

$$P_n = \frac{\sum_{C \in \{Candidates\}} \sum_{n\text{-}gram \in C} Count_{clip}(n\text{-}gram)}{\sum_{C' \in \{Candidates\}} \sum_{n\text{-}gram' \in C'} Count(n\text{-}gram')}$$

其中：
- $P_n$ 是n-gram精确率
- $Count(n\text{-}gram)$ 是n-gram在候选翻译中的出现次数
- $Count_{clip}(n\text{-}gram)$ 是n-gram在候选翻译中的出现次数，但不超过该n-gram在任何参考翻译中的最大出现次数

###  计算示例

假设有：
- 候选翻译："the the the the the the the"
- 参考翻译："the cat is on the mat"

对于unigram (1-gram)：
- 候选翻译中"the"出现7次
- 参考翻译中"the"出现2次
- 因此，$Count_{clip}("the") = \min(7, 2) = 2$
- $P_1 = \frac{2}{7} \approx 0.286$

###  长度惩罚
为了防止系统生成过短的翻译，BLEU引入了简短惩罚因子(Brevity Penalty, BP)：

$$BP = \begin{cases} 
1 & \text{if } c > r \\ 
e^{(1-r/c)} & \text{if } c \leq r 
\end{cases}$$

其中：
- $c$ 是候选翻译的长度
- $r$ 是参考翻译的长度（如果有多个参考翻译，选择与候选翻译长度最接近的那个）

###  BLEU分数计算
最终的BLEU分数结合了n-gram精确率和长度惩罚：

$$BLEU = BP \cdot \exp\left(\sum_{n=1}^{N} w_n \log P_n\right)$$

其中：
- $N$ 通常为4，表示考虑1-gram到4-gram
- $w_n$ 是权重，通常设为$w_n = 1/N$（均等权重）
- $BP$ 是长度惩罚因子

## 3. BLEU评分的计算步骤

###  计算流程

1. **分词**：将候选翻译和参考翻译分词
2. **计算n-gram**：统计候选翻译和参考翻译中各n-gram的出现次数
3. **计算修剪计数**：对候选翻译中的每个n-gram，计算其修剪计数$Count_{clip}$
4. **计算n-gram精确率**：计算各阶n-gram的精确率$P_1, P_2, P_3, P_4$
5. **计算长度惩罚**：根据候选翻译和参考翻译的长度计算BP
6. **计算BLEU分数**：结合n-gram精确率和长度惩罚计算最终分数

###  详细示例

假设有：
- 候选翻译："I am a student"
- 参考翻译1："I am a college student"
- 参考翻译2："I am a university student"

**步骤1-2**：分词并统计n-gram

候选翻译的n-gram：
- 1-gram: "I"(1), "am"(1), "a"(1), "student"(1)
- 2-gram: "I am"(1), "am a"(1), "a student"(1)
- 3-gram: "I am a"(1), "am a student"(1)
- 4-gram: "I am a student"(1)

**步骤3**：计算修剪计数

对于1-gram：
- "I"在参考翻译中最多出现1次，所以$Count_{clip}("I") = 1$
- "am"在参考翻译中最多出现1次，所以$Count_{clip}("am") = 1$
- "a"在参考翻译中最多出现1次，所以$Count_{clip}("a") = 1$
- "student"在参考翻译中最多出现1次，所以$Count_{clip}("student") = 1$

**步骤4**：计算n-gram精确率

- $P_1 = \frac{1+1+1+1}{4} = 1$
- $P_2 = \frac{1+1+1}{3} = 1$
- $P_3 = \frac{1+1}{2} = 1$
- $P_4 = \frac{0}{1} = 0$ (因为完整的4-gram在参考翻译中不存在)

**步骤5**：计算长度惩罚

- 候选翻译长度$c = 4$
- 参考翻译1长度$r_1 = 5$
- 参考翻译2长度$r_2 = 5$
- 选择$r = 5$（两者相同）
- $BP = e^{(1-5/4)} = e^{-0.25} \approx 0.779$

**步骤6**：计算BLEU分数

- $BLEU = 0.779 \cdot \exp(\frac{1}{4} \cdot (\log 1 + \log 1 + \log 1 + \log 0))$
- 由于$\log 0$是无定义的，实际计算中通常会进行平滑处理，或者只使用到3-gram
- 使用平滑后的值，最终BLEU分数约为0.7